In [1]:
import random
from math import exp

In [2]:
# test case we are choosing test case 1, use assert to check the answer is correct

## The init method initializes all parameters to zero. 

### If any parameter needs to have a default value, you can set it here. 

### All penalty parameters are set to a default value of 10

In [3]:
class PrimerDesign(object):

    def __init__(self, name):
        '''parameters for the length criterion'''
        self.max_length = 0
        self.min_length = 0
        self.penalty_length = 10

        '''parameters for the temperature difference criterion'''
        self.max_tdiff = 0
        self.min_tdiff = 0
        self.penalty_tdiff = 10

        '''parameters for the cg content criterion'''
        self.max_cg = 0
        self.min_cg = 0
        self.penalty_cg = 10

        '''parameters for the annealing temperature criterion'''
        self.max_temp = 0
        self.min_temp = 0
        self.penalty_temp = 10

        '''parameters for the run criterion'''
        self.run_threshold = 0
        self.penalty_runs = 10

        '''parameters for the repeat criterion'''
        self.repeat_threshold = 0
        self.penalty_repeats = 10

        '''parameters for the specificity criterion'''
        self.penalty_specificity = 10

        '''locations where the forward primer should be chosen from'''
        self.fp_start = 0
        self.fp_end = 0

        '''locations where the reverse primer should be chosen from'''
        self.rp_start = 0
        self.rp_end = 0

        ''' parameters for the simulated annealing portion'''
        self.initial_temperature = 200
        self.stopping_temperature = 0.01
        self.drop_fraction = 0.999
        
    def check(self,sq):
        if type(sq) != str:
            return "Error"
        else:
            ret = ''
            for i in sq:
                if i in ['a', 't', 'c', 'g']:
                    ret += i
            return ret

### Task 2 
set the dna sequence to be the sequence that we entered.

In [4]:
class PrimerDesign(PrimerDesign):
    def set_dna_sequence(self, dna_sequence):
        # check if the sequence if valid
        dna_sequence = self.check(dna_sequence)
        if dna_sequence == "Error":
            return "Error"
        else:
            # set the dna sequence to be the checked dna sequence
            self.dna_sequence = dna_sequence
            return dna_sequence

### Task 3
randam generate a index and use that index to generate a primer.

In [5]:
class PrimerDesign(PrimerDesign):
    def func_select_random(self, sqtype='forward', length=20):
        # if sqtype is forward, then random generate a index and cut the sequence
        if(sqtype == 'forward'):
            index = random.randint(self.fp_start, self.fp_end)
            self.forward_primer = self.dna_sequence[index:index+length]
            return self.forward_primer
        # if sqtype is bachward, then random generate a index and cut the sequence
        elif(sqtype == 'reverse'):
            index = random.randint(self.fp_start, self.fp_end)
            self.reverse_primer = self.dna_sequence[index:index+length]
            return self.reverse_primer
        else:
            return "Error"

### Task 4
count the length, temperature, cg fraction of the primer.

In [6]:
class PrimerDesign(PrimerDesign):
    def func_length(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:     
            # measure the length of the sequence
            return len(sq)

    def func_cg_fraction(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            # count the number of c and g in the sq
            if len(sq)>0:
                return (list(sq).count("c") + list(sq).count("g"))/len(sq)
            else:
                return 0

    def func_temperature(self, sq):
        sq = sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else: 
            # find out number of cg using func_length
            cg = self.func_length(sq) * self.func_cg_fraction(sq)
            # then at is length - number of cg
            at = self.func_length(sq) - cg
            temp = 4*cg+2*at
            return temp   

count the number of runs inside the sequence which is a squence of the same bases whose number is more than the threshold.

In [7]:
class PrimerDesign(PrimerDesign):
    def func_count_runs(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            if len(sq)==0:
                return 0
            else:
                # inicialize the variables for saving the data
                l, last, count = [], sq[0], 0
                for i in sq:
                    if i == last:
                        count += 1
                    else:
                        l.append(count)
                        count, last = 1, i
                # for the last count, check if it is counted inside
                if count>self.run_threshold:
                    count = 1
                else:
                    count = 0
                for i in l:
                    # check all that satisfies the threshold
                    if i > self.run_threshold:
                        count += 1
                return count

count the number of repeats in the sequence

In [8]:
class PrimerDesign(PrimerDesign):
    def func_count_repeats(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            di_repeats = ['at','ac','ag','ca','ct','cg','ga','gt','gc','ta','tc','tg']
            count = 0
            for i in range(1,len(sq)):
                if sq[i-1:i+1] == sq[i+1:i+3]:
                    if sq[i-1:i+1] in di_repeats:
                        count += 1
            return count 

### Task 5
The following methods implement the cost functions for each individual
criterion. They follow the mathematical expressions given in the handout

In [9]:
class PrimerDesign(PrimerDesign):

    def cost_length(self, sq):
        # check error input
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            # calculate cost length
            sq_len = len(sq)
            if(sq_len > self.max_length):
                return (sq_len - self.max_length)*self.penalty_length
            elif(sq_len > self.min_length):
                return 0
            else:
                return (self.min_length - sq_len)*self.penalty_length

    def cost_temperature(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            # calculate cost temperature
            t = self.func_temperature(sq)
            if t > self.max_temp:
                return (t - self.max_temp) * self.penalty_temp
            elif t >= self.min_temp:
                return 0
            else:
                return (self.min_temp - t) * self.penalty_temp

    def cost_cgcontent(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            # calculate cost cg 
            cg = self.func_cg_fraction(sq)
            if cg > 0.6:
                return self.penalty_cg * (cg - 0.6)
            elif cg >= 0.4:
                return 0
            else:
                return self.penalty_cg * (0.4 - cg)

    def cost_temperature_difference(self, fp, rp):
        fp = self.check(fp)
        rp = self.check(rp)
        if fp == "Error" or rp == "Error":
            return "Error"
        else:
            # calculate cost temperature difference
            fpt = self.func_temperature(fp)
            rpt = self.func_temperature(rp)
            delta_t = abs(fpt - rpt)
            if delta_t > self.max_tdiff:
                return self.penalty_tdiff * (delta_t - self.max_tdiff)
            else:
                return 0

    def cost_specificity(self, fp):
        fp = self.check(fp)
        if fp == "Error":
            return "Error"
        else:
            # find out same fp inside the sequence
            return self.penalty_specificity*(self.dna_sequence.count(fp)-1)

    def cost_runs(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            # count the number of same base come out together
            runs = self.func_count_runs(sq)
            return runs*self.penalty_runs

    def cost_repeats(self, sq):
        sq = self.check(sq)
        if sq == "Error":
            return "Error"
        else:
            # cost of the repeat
            repeats = self.func_count_repeats(sq)
            return self.penalty_repeats*repeats

### Task 6
summing up all the cases of all of the criterions and return the sum of all of the cost.

In [10]:
class PrimerDesign(PrimerDesign):
    # adding up all the cost together
    def cost_objective_function(self, fp, rp):
        '''complete the calculation of the cost'''
        cost = self.cost_length(fp) + self.cost_length(rp)
        +self.cost_temperature(fp) + self.cost_temperature(rp)
        +self.cost_cgcontent(fp) + self.cost_cgcontent(rp)
        +self.cost_temperature_difference(fp, rp)
        +self.cost_specificity(fp) + self.cost_specificity(rp)
        +self.cost_runs(fp) + self.cost_runs(rp)
        +self.cost_repeats(fp) + self.cost_repeats(rp)
        return cost 

### Task 7
give a summery of what these two primers are like whether they meet the requirement.

In [11]:
class PrimerDesign(PrimerDesign):
    # print out the information of fp and rp respectively and format it
    def cost_objective_function_info(self, fp, rp):
        ret = "===Forward Primer=== {0}\n".format(fp)+ \
        "{0:<25}{1:>25}{2:>5}\n".format("Criterion","Cost function score","Pass")+ \
        "-"*55+"\n" + \
        "{0:<25}{1:>25}{2:>5}\n".format("length",self.cost_length(fp),"Yes" if self.cost_length(fp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("annealing temperature",self.cost_temperature(fp),"Yes" if self.cost_temperature(fp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("%cg_content",self.cost_cgcontent(fp),"Yes" if self.cost_cgcontent(fp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("specificity",self.cost_specificity(fp),"Yes" if self.cost_specificity(fp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("runs",self.cost_runs(fp),"Yes" if self.cost_runs(fp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("repeats",self.cost_repeats(fp),"Yes" if self.cost_repeats(fp)==0 else "No") + \
        " "*55 + "\n" + \
        "===Forward Primer=== {0}\n".format(rp)+ \
        "{0:<25}{1:>25}{2:>5}\n".format("Criterion","Cost function score","Pass")+ \
        "-"*55+"\n" + \
        "{0:<25}{1:>25}{2:>5}\n".format("length",self.cost_length(rp),"Yes" if self.cost_length(rp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("annealing temperature",self.cost_temperature(rp),"Yes" if self.cost_temperature(rp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("%cg_content",self.cost_cgcontent(rp),"Yes" if self.cost_cgcontent(rp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("specificity",self.cost_specificity(rp),"Yes" if self.cost_specificity(rp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("runs",self.cost_runs(rp),"Yes" if self.cost_runs(rp)==0 else "No") + \
        "{0:<25}{1:>25}{2:>5}\n".format("repeats",self.cost_repeats(rp),"Yes" if self.cost_repeats(rp)==0 else "No") + \
        " "*55 + "\n" + \
        "{0:<25}{1:>25}{2:>5}\n".format("Temperature Difference",self.cost_temperature_difference(fp,rp),"Yes" if self.cost_temperature_difference(fp,rp)==0 else "No")
        return ret

##### Task 10
the function below will generate the sequence of forward and reverse, check if it meets the requirement, updata until the primers meets the requirements.

In [12]:
class PrimerDesign(PrimerDesign): 
    def get_primer_neighbour(self,sq):
        # generate a random length between the max length and the minimum length
        length = random.randint(self.min_length,self.max_length)
        # generate a number for shift
        new = random.randint(-20,20)
        # find out the current index
        start_position = self.dna_sequence.index(sq)
        # current index adding the shift get the new index
        new_start_position = start_position + new
        new_end_positiion = start_position + length
        # generate a new sequence
        return self.dna_sequence[new_start_position:new_end_positiion]
        
    
    def func_simulated_annealing(self):
        # initialize the parameters
        temperature = self.initial_temperature
        stopping_temperature = self.stopping_temperature
        drop = self.drop_fraction
        # generate the random index for fp and rp
        fp_current = self.func_select_random("forward",20)
        rp_current = self.func_select_random("reverse",20)
        # get the current cost for the pair of fp and rp
        print(fp_current,rp_current)
        current_cost = self.cost_objective_function(fp_current,rp_current)
        
        while temperature > stopping_temperature:
            # generate two base that is nearing the current one
            fp_new = self.get_primer_neighbour(fp_current)
            rp_new = self.get_primer_neighbour(rp_current)
            # get the new cost for the pair of fp and rp
            new_cost = self.cost_objective_function(fp_new,rp_new)
            # update fp and rp if the cost is smaller
            if new_cost < current_cost:
                fp_current = fp_new
                rp_current = rp_new
                current_cost = new_cost
            else:
                # random generate a threshold to decide if we shall keep the new fp and rp
                delta = new_cost - current_cost
                acceptance_probability = exp(-delta/temperature)
                num = random.random()
                # update fp and rp
                if (acceptance_probability>num):
                    fp_current = fp_new
                    rp_current = rp_new
                    current_cost = new_cost
            # update temperature
            temperature = temperature*drop
        return fp_current,rp_current

### Store the DNA sequence given to you in the variable below 

In [13]:
dna_sequence = """1 gtggcgcgag cttctgaaac taggcggcag aggcggagcc gctgtggcac tgctgcgcct
61 ctgctgcgcc tcgggtgtct tttgcggcgg tgggtcgccg ccgggagaag cgtgagggga
121 cagatttgtg accggcgcgg tttttgtcag cttactccgg ccaaaaaaga actgcacctc
181 tggagcggac ttatttacca agcattggag gaatatcgta ggtaaaaatg cctattggat
241 ccaaagagag gccaacattt tttgaaattt ttaagacacg ctgcaacaaa gcagatttag
301 gaccaataag tcttaattgg tttgaagaac tttcttcaga agctccaccc tataattctg
361 aacctgcaga agaatctgaa cataaaaaca acaattacga accaaaccta tttaaaactc
421 cacaaaggaa accatcttat aatcagctgg cttcaactcc aataatattc aaagagcaag
481 ggctgactct gccgctgtac caatctcctg taaaagaatt agataaattc aaattagact
541 taggaaggaa tgttcccaat agtagacata aaagtcttcg cacagtgaaa actaaaatgg
601 atcaagcaga tgatgtttcc tgtccacttc taaattcttg tcttagtgaa agtcctgttg
661 ttctacaatg tacacatgta acaccacaaa gagataagtc agtggtatgt gggagtttgt
721 ttcatacacc aaagtttgtg aagggtcgtc agacaccaaa acatatttct gaaagtctag
781 gagctgaggt ggatcctgat atgtcttggt caagttcttt agctacacca cccaccctta
841 gttctactgt gctcatagtc agaaatgaag aagcatctga aactgtattt cctcatgata
901 ctactgctaa tgtgaaaagc tatttttcca atcatgatga aagtctgaag aaaaatgata
961 gatttatcgc ttctgtgaca gacagtgaaa acacaaatca aagagaagct gcaagtcatg
1021 gatttggaaa aacatcaggg aattcattta aagtaaatag ctgcaaagac cacattggaa
1081 agtcaatgcc aaatgtccta gaagatgaag tatatgaaac agttgtagat acctctgaag
1141 aagatagttt ttcattatgt ttttctaaat gtagaacaaa aaatctacaa aaagtaagaa
1201 ctagcaagac taggaaaaaa attttccatg aagcaaacgc tgatgaatgt gaaaaatcta
1261 aaaaccaagt gaaagaaaaa tactcatttg tatctgaagt ggaaccaaat gatactgatc
1321 cattagattc aaatgtagca aatcagaagc cctttgagag tggaagtgac aaaatctcca
1381 aggaagttgt accgtctttg gcctgtgaat ggtctcaact aaccctttca ggtctaaatg
1441 gagcccagat ggagaaaata cccctattgc atatttcttc atgtgaccaa aatatttcag
1501 aaaaagacct attagacaca gagaacaaaa gaaagaaaga ttttcttact tcagagaatt
1561 ctttgccacg tatttctagc ctaccaaaat cagagaagcc attaaatgag gaaacagtgg
1621 taaataagag agatgaagag cagcatcttg aatctcatac agactgcatt cttgcagtaa
1681 agcaggcaat atctggaact tctccagtgg cttcttcatt tcagggtatc aaaaagtcta
1741 tattcagaat aagagaatca cctaaagaga ctttcaatgc aagtttttca ggtcatatga
1801 ctgatccaaa ctttaaaaaa gaaactgaag cctctgaaag tggactggaa atacatactg
1861 tttgctcaca gaaggaggac tccttatgtc caaatttaat tgataatgga agctggccag
1921 ccaccaccac acagaattct gtagctttga agaatgcagg tttaatatcc actttgaaaa
1981 agaaaacaaa taagtttatt tatgctatac atgatgaaac atcttataaa ggaaaaaaaa
2041 taccgaaaga ccaaaaatca gaactaatta actgttcagc ccagtttgaa gcaaatgctt
2101 ttgaagcacc acttacattt gcaaatgctg attcaggttt attgcattct tctgtgaaaa
2161 gaagctgttc acagaatgat tctgaagaac caactttgtc cttaactagc tcttttggga
2221 caattctgag gaaatgttct agaaatgaaa catgttctaa taatacagta atctctcagg
2281 atcttgatta taaagaagca aaatgtaata aggaaaaact acagttattt attaccccag
2341 aagctgattc tctgtcatgc ctgcaggaag gacagtgtga aaatgatcca aaaagcaaaa
2401 aagtttcaga tataaaagaa gaggtcttgg ctgcagcatg tcacccagta caacattcaa
2461 aagtggaata cagtgatact gactttcaat cccagaaaag tcttttatat gatcatgaaa
2521 atgccagcac tcttatttta actcctactt ccaaggatgt tctgtcaaac ctagtcatga
2581 tttctagagg caaagaatca tacaaaatgt cagacaagct caaaggtaac aattatgaat
2641 ctgatgttga attaaccaaa aatattccca tggaaaagaa tcaagatgta tgtgctttaa
2701 atgaaaatta taaaaacgtt gagctgttgc cacctgaaaa atacatgaga gtagcatcac
2761 cttcaagaaa ggtacaattc aaccaaaaca caaatctaag agtaatccaa aaaaatcaag
2821 aagaaactac ttcaatttca aaaataactg tcaatccaga ctctgaagaa cttttctcag
2881 acaatgagaa taattttgtc ttccaagtag ctaatgaaag gaataatctt gctttaggaa
2941 atactaagga acttcatgaa acagacttga cttgtgtaaa cgaacccatt ttcaagaact
3001 ctaccatggt tttatatgga gacacaggtg ataaacaagc aacccaagtg tcaattaaaa
3061 aagatttggt ttatgttctt gcagaggaga acaaaaatag tgtaaagcag catataaaaa
3121 tgactctagg tcaagattta aaatcggaca tctccttgaa tatagataaa ataccagaaa
3181 aaaataatga ttacatgaac aaatgggcag gactcttagg tccaatttca aatcacagtt
3241 ttggaggtag cttcagaaca gcttcaaata aggaaatcaa gctctctgaa cataacatta
3301 agaagagcaa aatgttcttc aaagatattg aagaacaata tcctactagt ttagcttgtg
3361 ttgaaattgt aaataccttg gcattagata atcaaaagaa actgagcaag cctcagtcaa
3421 ttaatactgt atctgcacat ttacagagta gtgtagttgt ttctgattgt aaaaatagtc
3481 atataacccc tcagatgtta ttttccaagc aggattttaa ttcaaaccat aatttaacac
3541 ctagccaaaa ggcagaaatt acagaacttt ctactatatt agaagaatca ggaagtcagt
3601 ttgaatttac tcagtttaga aaaccaagct acatattgca gaagagtaca tttgaagtgc
3661 ctgaaaacca gatgactatc ttaaagacca cttctgagga atgcagagat gctgatcttc
3721 atgtcataat gaatgcccca tcgattggtc aggtagacag cagcaagcaa tttgaaggta
3781 cagttgaaat taaacggaag tttgctggcc tgttgaaaaa tgactgtaac aaaagtgctt
3841 ctggttattt aacagatgaa aatgaagtgg ggtttagggg cttttattct gctcatggca
3901 caaaactgaa tgtttctact gaagctctgc aaaaagctgt gaaactgttt agtgatattg
3961 agaatattag tgaggaaact tctgcagagg tacatccaat aagtttatct tcaagtaaat
4021 gtcatgattc tgttgtttca atgtttaaga tagaaaatca taatgataaa actgtaagtg
4081 aaaaaaataa taaatgccaa ctgatattac aaaataatat tgaaatgact actggcactt
4141 ttgttgaaga aattactgaa aattacaaga gaaatactga aaatgaagat aacaaatata
4201 ctgctgccag tagaaattct cataacttag aatttgatgg cagtgattca agtaaaaatg
4261 atactgtttg tattcataaa gatgaaacgg acttgctatt tactgatcag cacaacatat
4321 gtcttaaatt atctggccag tttatgaagg agggaaacac tcagattaaa gaagatttgt
4381 cagatttaac ttttttggaa gttgcgaaag ctcaagaagc atgtcatggt aatacttcaa
4441 ataaagaaca gttaactgct actaaaacgg agcaaaatat aaaagatttt gagacttctg
4501 atacattttt tcagactgca agtgggaaaa atattagtgt cgccaaagag tcatttaata
4561 aaattgtaaa tttctttgat cagaaaccag aagaattgca taacttttcc ttaaattctg
4621 aattacattc tgacataaga aagaacaaaa tggacattct aagttatgag gaaacagaca
4681 tagttaaaca caaaatactg aaagaaagtg tcccagttgg tactggaaat caactagtga
4741 ccttccaggg acaacccgaa cgtgatgaaa agatcaaaga acctactcta ttgggttttc
4801 atacagctag cgggaaaaaa gttaaaattg caaaggaatc tttggacaaa gtgaaaaacc
4861 tttttgatga aaaagagcaa ggtactagtg aaatcaccag ttttagccat caatgggcaa
4921 agaccctaaa gtacagagag gcctgtaaag accttgaatt agcatgtgag accattgaga
4981 tcacagctgc cccaaagtgt aaagaaatgc agaattctct caataatgat aaaaaccttg
5041 tttctattga gactgtggtg ccacctaagc tcttaagtga taatttatgt agacaaactg
5101 aaaatctcaa aacatcaaaa agtatctttt tgaaagttaa agtacatgaa aatgtagaaa
5161 aagaaacagc aaaaagtcct gcaacttgtt acacaaatca gtccccttat tcagtcattg
5221 aaaattcagc cttagctttt tacacaagtt gtagtagaaa aacttctgtg agtcagactt
5281 cattacttga agcaaaaaaa tggcttagag aaggaatatt tgatggtcaa ccagaaagaa
5341 taaatactgc agattatgta ggaaattatt tgtatgaaaa taattcaaac agtactatag
5401 ctgaaaatga caaaaatcat ctctccgaaa aacaagatac ttatttaagt aacagtagca
5461 tgtctaacag ctattcctac cattctgatg aggtatataa tgattcagga tatctctcaa
5521 aaaataaact tgattctggt attgagccag tattgaagaa tgttgaagat caaaaaaaca
5581 ctagtttttc caaagtaata tccaatgtaa aagatgcaaa tgcataccca caaactgtaa
5641 atgaagatat ttgcgttgag gaacttgtga ctagctcttc accctgcaaa aataaaaatg
5701 cagccattaa attgtccata tctaatagta ataattttga ggtagggcca cctgcattta
5761 ggatagccag tggtaaaatc gtttgtgttt cacatgaaac aattaaaaaa gtgaaagaca
5821 tatttacaga cagtttcagt aaagtaatta aggaaaacaa cgagaataaa tcaaaaattt
5881 gccaaacgaa aattatggca ggttgttacg aggcattgga tgattcagag gatattcttc
5941 ataactctct agataatgat gaatgtagca cgcattcaca taaggttttt gctgacattc
6001 agagtgaaga aattttacaa cataaccaaa atatgtctgg attggagaaa gtttctaaaa
6061 tatcaccttg tgatgttagt ttggaaactt cagatatatg taaatgtagt atagggaagc
6121 ttcataagtc agtctcatct gcaaatactt gtgggatttt tagcacagca agtggaaaat
6181 ctgtccaggt atcagatgct tcattacaaa acgcaagaca agtgttttct gaaatagaag
6241 atagtaccaa gcaagtcttt tccaaagtat tgtttaaaag taacgaacat tcagaccagc
6301 tcacaagaga agaaaatact gctatacgta ctccagaaca tttaatatcc caaaaaggct
6361 tttcatataa tgtggtaaat tcatctgctt tctctggatt tagtacagca agtggaaagc
6421 aagtttccat tttagaaagt tccttacaca aagttaaggg agtgttagag gaatttgatt
6481 taatcagaac tgagcatagt cttcactatt cacctacgtc tagacaaaat gtatcaaaaa
6541 tacttcctcg tgttgataag agaaacccag agcactgtgt aaactcagaa atggaaaaaa
6601 cctgcagtaa agaatttaaa ttatcaaata acttaaatgt tgaaggtggt tcttcagaaa
6661 ataatcactc tattaaagtt tctccatatc tctctcaatt tcaacaagac aaacaacagt
6721 tggtattagg aaccaaagtg tcacttgttg agaacattca tgttttggga aaagaacagg
6781 cttcacctaa aaacgtaaaa atggaaattg gtaaaactga aactttttct gatgttcctg
6841 tgaaaacaaa tatagaagtt tgttctactt actccaaaga ttcagaaaac tactttgaaa
6901 cagaagcagt agaaattgct aaagctttta tggaagatga tgaactgaca gattctaaac
6961 tgccaagtca tgccacacat tctcttttta catgtcccga aaatgaggaa atggttttgt
7021 caaattcaag aattggaaaa agaagaggag agccccttat cttagtggga gaaccctcaa
7081 tcaaaagaaa cttattaaat gaatttgaca ggataataga aaatcaagaa aaatccttaa
7141 aggcttcaaa aagcactcca gatggcacaa taaaagatcg aagattgttt atgcatcatg
7201 tttctttaga gccgattacc tgtgtaccct ttcgcacaac taaggaacgt caagagatac
7261 agaatccaaa ttttaccgca cctggtcaag aatttctgtc taaatctcat ttgtatgaac
7321 atctgacttt ggaaaaatct tcaagcaatt tagcagtttc aggacatcca ttttatcaag
7381 tttctgctac aagaaatgaa aaaatgagac acttgattac tacaggcaga ccaaccaaag
7441 tctttgttcc accttttaaa actaaatcac attttcacag agttgaacag tgtgttagga
7501 atattaactt ggaggaaaac agacaaaagc aaaacattga tggacatggc tctgatgata
7561 gtaaaaataa gattaatgac aatgagattc atcagtttaa caaaaacaac tccaatcaag
7621 cagcagctgt aactttcaca aagtgtgaag aagaaccttt agatttaatt acaagtcttc
7681 agaatgccag agatatacag gatatgcgaa ttaagaagaa acaaaggcaa cgcgtctttc
7741 cacagccagg cagtctgtat cttgcaaaaa catccactct gcctcgaatc tctctgaaag
7801 cagcagtagg aggccaagtt ccctctgcgt gttctcataa acagctgtat acgtatggcg
7861 tttctaaaca ttgcataaaa attaacagca aaaatgcaga gtcttttcag tttcacactg
7921 aagattattt tggtaaggaa agtttatgga ctggaaaagg aatacagttg gctgatggtg
7981 gatggctcat accctccaat gatggaaagg ctggaaaaga agaattttat agggctctgt
8041 gtgacactcc aggtgtggat ccaaagctta tttctagaat ttgggtttat aatcactata
8101 gatggatcat atggaaactg gcagctatgg aatgtgcctt tcctaaggaa tttgctaata
8161 gatgcctaag cccagaaagg gtgcttcttc aactaaaata cagatatgat acggaaattg
8221 atagaagcag aagatcggct ataaaaaaga taatggaaag ggatgacaca gctgcaaaaa
8281 cacttgttct ctgtgtttct gacataattt cattgagcgc aaatatatct gaaacttcta
8341 gcaataaaac tagtagtgca gatacccaaa aagtggccat tattgaactt acagatgggt
8401 ggtatgctgt taaggcccag ttagatcctc ccctcttagc tgtcttaaag aatggcagac
8461 tgacagttgg tcagaagatt attcttcatg gagcagaact ggtgggctct cctgatgcct
8521 gtacacctct tgaagcccca gaatctctta tgttaaagat ttctgctaac agtactcggc
8581 ctgctcgctg gtataccaaa cttggattct ttcctgaccc tagacctttt cctctgccct
8641 tatcatcgct tttcagtgat ggaggaaatg ttggttgtgt tgatgtaatt attcaaagag
8701 cataccctat acagtggatg gagaagacat catctggatt atacatattt cgcaatgaaa
8761 gagaggaaga aaaggaagca gcaaaatatg tggaggccca acaaaagaga ctagaagcct
8821 tattcactaa aattcaggag gaatttgaag aacatgaaga aaacacaaca aaaccatatt
8881 taccatcacg tgcactaaca agacagcaag ttcgtgcttt gcaagatggt gcagagcttt
8941 atgaagcagt gaagaatgca gcagacccag cttaccttga gggttatttc agtgaagagc
9001 agttaagagc cttgaataat cacaggcaaa tgttgaatga taagaaacaa gctcagatcc
9061 agttggaaat taggaaggcc atggaatctg ctgaacaaaa ggaacaaggt ttatcaaggg
9121 atgtcacaac cgtgtggaag ttgcgtattg taagctattc aaaaaaagaa aaagattcag
9181 ttatactgag tatttggcgt ccatcatcag atttatattc tctgttaaca gaaggaaaga
9241 gatacagaat ttatcatctt gcaacttcaa aatctaaaag taaatctgaa agagctaaca
9301 tacagttagc agcgacaaaa aaaactcagt atcaacaact accggtttca gatgaaattt
9361 tatttcagat ttaccagcca cgggagcccc ttcacttcag caaattttta gatccagact
9421 ttcagccatc ttgttctgag gtggacctaa taggatttgt cgtttctgtt gtgaaaaaaa
9481 caggacttgc ccctttcgtc tatttgtcag acgaatgtta caatttactg gcaataaagt
9541 tttggataga ccttaatgag gacattatta agcctcatat gttaattgct gcaagcaacc
9601 tccagtggcg accagaatcc aaatcaggcc ttcttacttt atttgctgga gatttttctg
9661 tgttttctgc tagtccaaaa gagggccact ttcaagagac attcaacaaa atgaaaaata
9721 ctgttgagaa tattgacata ctttgcaatg aagcagaaaa caagcttatg catatactgc
9781 atgcaaatga tcccaagtgg tccaccccaa ctaaagactg tacttcaggg ccgtacactg
9841 ctcaaatcat tcctggtaca ggaaacaagc ttctgatgtc ttctcctaat tgtgagatat
9901 attatcaaag tcctttatca ctttgtatgg ccaaaaggaa gtctgtttcc acacctgtct
9961 cagcccagat gacttcaaag tcttgtaaag gggagaaaga gattgatgac caaaagaact
10021 gcaaaaagag aagagccttg gatttcttga gtagactgcc tttacctcca cctgttagtc
10081 ccatttgtac atttgtttct ccggctgcac agaaggcatt tcagccacca aggagttgtg
10141 gcaccaaata cgaaacaccc ataaagaaaa aagaactgaa ttctcctcag atgactccat
10201 ttaaaaaatt caatgaaatt tctcttttgg aaagtaattc aatagctgac gaagaacttg
10261 cattgataaa tacccaagct cttttgtctg gttcaacagg agaaaaacaa tttatatctg
10321 tcagtgaatc cactaggact gctcccacca gttcagaaga ttatctcaga ctgaaacgac
10381 gttgtactac atctctgatc aaagaacagg agagttccca ggccagtacg gaagaatgtg
10441 agaaaaataa gcaggacaca attacaacta aaaaatatat ctaagcattt gcaaaggcga
10501 caataaatta ttgacgctta acctttccag tttataagac tggaatataa tttcaaacca
10561 cacattagta cttatgttgc acaatgagaa aagaaattag tttcaaattt acctcagcgt
10621 ttgtgtatcg ggcaaaaatc gttttgcccg attccgtatt ggtatacttt tgcttcagtt
10681 gcatatctta aaactaaatg taatttatta actaatcaag aaaaacatct ttggctgagc
10741 tcggtggctc atgcctgtaa tcccaacact ttgagaagct gaggtgggag gagtgcttga
10801 ggccaggagt tcaagaccag cctgggcaac atagggagac ccccatcttt acaaagaaaa
10861 aaaaaagggg aaaagaaaat cttttaaatc tttggatttg atcactacaa gtattatttt
10921 acaagtgaaa taaacatacc attttctttt agattgtgtc attaaatgga atgaggtctc
10981 ttagtacagt tattttgatg cagataattc cttttagttt agctactatt ttaggggatt
11041 ttttttagag gtaactcact atgaaatagt tctccttaat gcaaatatgt tggttctgct
11101 atagttccat cctgttcaaa agtcaggatg aatatgaaga gtggtgtttc cttttgagca
11161 attcttcatc cttaagtcag catgattata agaaaaatag aaccctcagt gtaactctaa
11221 ttccttttta ctattccagt gtgatctctg aaattaaatt acttcaacta aaaattcaaa
11281 tactttaaat cagaagattt catagttaat ttattttttt tttcaacaaa atggtcatcc
11341 aaactcaaac ttgagaaaat atcttgcttt caaattggca ctgatt
"""

### Instantiate your class and read in the DNA sequence

the test cases are:

check if we lost one letter of forward func_select_random will work or not

use "forwar" instead of "forward"

check if we enter a list instead of a string func_length will work or not

use list("agtcttg") instead of a string

check if we enter a number for the func_temperature if func_temperture will work or not

direct give the function a temperature

check if we add other letters and numbers to the sequence, function func_cg_fraction will work or not

check if we add more letters func_count_runs and function repeat can work

use sequence "atgaa123sdggtaagcatagafgtag"

In [14]:
test = PrimerDesign(dna_sequence)
# test task 2
print("\nTask 2")
dna_sequence = test.set_dna_sequence(dna_sequence)
# print("the DNA sequence is:",dna_sequence)

# test task 3
print("\ntask 3")
# assign self.fp_start and self.fp_end,run_threshold
test.fp_start = 100
test.fp_end = 200
test.run_threshold = 4
# random select one forward primer and one reverse primer
forward = test.func_select_random("forward",19)
print("The forward primer is:{0}".format(forward))

print("\ntask 4")
# count the length of the primers
fp_length = test.func_length(forward)
print("the forward primer has a length of:",fp_length)

# the annealing temperature 
fp_temp = test.func_temperature(forward)
print("the temperature of the forward primer is:",fp_temp)

# the fraction of cg bases
fp_cg = test.func_cg_fraction(forward)
print("the cg fraction of the forward primer is:",fp_cg)

# the numbers of runs present
fp_runs = test.func_count_runs(forward)
print("the runs of the forward primer is:",fp_runs)

# the numbers of repeats present
fp_repeats = test.func_count_repeats(forward)
print("the repeats of the forward primer is:",fp_repeats)

print("\ntask 6")
# cost for length
fp_length_cost = test.cost_length(forward)
print("the cost for the length is:",fp_length_cost)

# cost for temperature
fp_temperature_cost = test.cost_temperature(forward)
print("the cost for the temperature is:",fp_temperature_cost)

# cost for cg fraction
fp_cg_cost = test.cost_cgcontent(forward)
print("the cost for the cg fraction is:",fp_cg_cost)

# cost for the number of runs
fp_runs_cost = test.cost_runs(forward)
print("the cost for the cg runs is:",fp_runs_cost)

# cost for the number of repeats
fp_repeats_cost = test.cost_repeats(forward)
print("the cost for the cg repeats is:",fp_repeats_cost)

# task 5
print("\ntask 5")
print("primer=",test.func_select_random("forwar",19))
assert(test.func_select_random("forwar",19)=="Error")
print("length=",test.func_length(list("agtcttg")))
assert(test.func_length(list("agtcttg"))=="Error")
print("temperature=",test.func_temperature(35))
assert(test.func_temperature(35)=="Error")
print("cg fraction=",test.func_cg_fraction("atgaa123sdggtaagcatagafgtag"))
assert(test.func_cg_fraction("atgaa123sdggtaagcatagafgtag")-0.38095238095238093==0)
print("runs=",test.func_count_runs("atgaa123sdggtaagcatagafgtag"))
assert(test.func_count_runs("atgaa123sdggtaagcatagafgtag")==0)
print("repeats=",test.func_count_repeats("atgaa123sdggtaagcatagafgtag"))
assert(test.func_count_repeats("atgaa123sdggtaagcatagafgtag")==1)

# task 8
print("\ntask 8")
reverse = test.func_select_random("reverse",19)
cost = test.cost_objective_function(forward,reverse)
print("the single score for the cost is:",cost)
print("the cost info for the two primers")
print(test.cost_objective_function_info(forward,reverse))


Task 2

task 3
The forward primer is:cggccaaaaaagaactgca

task 4
the forward primer has a length of: 19
the temperature of the forward primer is: 56.0
the cg fraction of the forward primer is: 0.47368421052631576
the runs of the forward primer is: 1
the repeats of the forward primer is: 0

task 6
the cost for the length is: 190
the cost for the temperature is: 560.0
the cost for the cg fraction is: 0
the cost for the cg runs is: 10
the cost for the cg repeats is: 0

task 5
primer= Error
length= Error
temperature= Error
cg fraction= 0.38095238095238093
runs= 0
repeats= 1

task 8
the single score for the cost is: 380
the cost info for the two primers
===Forward Primer=== cggccaaaaaagaactgca
Criterion                      Cost function score Pass
-------------------------------------------------------
length                                         190   No
annealing temperature                        560.0   No
%cg_content                                      0  Yes
specificity          

In [15]:
# test cases using test case 1

# tast 7 test cases
test.run_threshold = 4
forward = "cattaaaaatacgaaaaaagtcat"
print("\ntask 4")
# count the length of the primers
fp_length = test.func_length(forward)
print("the forward primer has a length of:",fp_length)

# the annealing temperature 
fp_temp = test.func_temperature(forward)
print("the temperature of the forward primer is:",fp_temp)

# the fraction of cg bases
fp_cg = test.func_cg_fraction(forward)
print("the cg fraction of the forward primer is:",fp_cg)

# the numbers of runs present
fp_runs = test.func_count_runs(forward)
print("the runs of the forward primer is:",fp_runs)

# the numbers of repeats present
fp_repeats = test.func_count_repeats(forward)
print("the repeats of the forward primer is:",fp_repeats)

print("\ntask 6")
# cost for length
fp_length_cost = test.cost_length(forward)
print("the cost for the length is:",fp_length_cost)

# cost for temperature
fp_temperature_cost = test.cost_temperature(forward)
print("the cost for the temperature is:",fp_temperature_cost)

# cost for cg fraction
fp_cg_cost = test.cost_cgcontent(forward)
print("the cost for the cg fraction is:",fp_cg_cost)

# cost for the number of runs
fp_runs_cost = test.cost_runs(forward)
print("the cost for the cg runs is:",fp_runs_cost)

# cost for the number of repeats
fp_repeats_cost = test.cost_repeats(forward)
print("the cost for the cg repeats is:",fp_repeats_cost)

# check if all the cases are correct
assert(fp_length == 24)
assert(fp_temp == 58)
assert(fp_cg == 5/24)
assert(fp_runs == 2)
assert(fp_repeats == 0)


task 4
the forward primer has a length of: 24
the temperature of the forward primer is: 58.0
the cg fraction of the forward primer is: 0.20833333333333334
the runs of the forward primer is: 2
the repeats of the forward primer is: 0

task 6
the cost for the length is: 240
the cost for the temperature is: 580.0
the cost for the cg fraction is: 1.9166666666666667
the cost for the cg runs is: 20
the cost for the cg repeats is: 0


### If you need to adjust any parameter from their default values in the init method, do it here

In [16]:
test.max_length = 22
test.min_length = 18
test.max_tdiff = 10
test.min_tdiff = 4
test.max_cg = 0.6
test.min_cg = 0.4
test.max_temp = 50
test.min_temp = 40
test.run_threshold = 4
test.repeat_threshold = 2
test.fp_start = 100
test.fp_end = 300
test.rp_start = 400
test.rp_end = 500
test.initial_temperature = 200
test.stopping_temperature = 0.01
test.drop_fraction = 0.999

### Show the outcome of your testing and the functions in the subsequent cells 

In [38]:
random.seed(20)
forward,reverse = test.func_simulated_annealing()
print(test.cost_objective_function_info(forward,reverse))

acaaagcagatttaggacca acacgctgcaacaaagcaga
===Forward Primer=== gcagaggcggagccgctgtgg
Criterion                      Cost function score Pass
-------------------------------------------------------
length                                           0  Yes
annealing temperature                        240.0   No
%cg_content                     1.6190476190476188   No
specificity                                      0  Yes
runs                                             0  Yes
repeats                                         20   No
                                                       
===Forward Primer=== cgagcttctgaaactaggcgg
Criterion                      Cost function score Pass
-------------------------------------------------------
length                                           0  Yes
annealing temperature                        160.0   No
%cg_content                                      0  Yes
specificity                                      0  Yes
runs                            